In [ ]:
# Sleeps are used because these notebooks are being tested automatically as part of the CI/CD. 
# In a normal user interaction, these sleeps would not be necessary.
from time import sleep

In [ ]:
def connect_zambeze_queue():
    import pika
    connection = pika.BlockingConnection(
        pika.ConnectionParameters('localhost', 5672)
    )
    channel = connection.channel()
    channel.queue_declare('hello')
    return channel

def send_zambeze_message(channel):
    from uuid import uuid4
    import json
    another_act_id = str(uuid4())
    act_id = str(uuid4())
    msg = {
            "name": "ImageMagick",
            "activity_id": act_id,
            "campaign_id": "campaign-uuid",
            "origin_agent_id": "def-uuid",
            "files": ["globus://Users/6o1/file.txt"],
            "command": "convert",
            "activity_status": "CREATED",
            "arguments": [
                "-delay",
                "20",
                "-loop",
                "0",
                "~/tests/campaigns/imagesequence/*.jpg",
                "a.gif",
            ],
            "kwargs": {},
            "depends_on": [another_act_id],
    }
    channel.basic_publish(
        exchange="",
        routing_key="hello",
        body=json.dumps(msg),
    )
    return act_id

In [ ]:
# Set log levels
import os
os.environ['LOG_STREAM_LEVEL'] = "error"
os.environ['LOG_FILE_LEVEL'] = "debug"

## Start Zambeze Flowceptor

In [ ]:
from flowcept import ZambezeInterceptor
interceptor = ZambezeInterceptor()

In [ ]:
from flowcept import FlowceptConsumerAPI
consumer = FlowceptConsumerAPI(interceptor)
consumer.start()

## Send Zambeze Message

In [ ]:
channel = connect_zambeze_queue()

In [ ]:
act_id = send_zambeze_message(channel)
act_id

In [ ]:
sleep(10)

## Check the task in Flowcept's database

In [ ]:
from flowcept import TaskQueryAPI
query_api = TaskQueryAPI()

In [ ]:
_filter = {"task_id": act_id}
query_api.query(_filter)

## Stop consumer

In [ ]:
consumer.stop()